## Loading data

In [2]:
import sys
import networkx as nx # type: ignore
import graphviz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dowhy import CausalModel
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
sys.path.append('../')
from src.data.make_dataset import load_data, merge_all_datasets
#from src.data.preprocess import handle_missing_values, rolling_mean_customer, rolling_mean_seller, haversine, preprocessing
from src.data.preprocess import HandleMissingValues, ProcessingHelpers, Preprocess


In [3]:
state_to_region = {
    'AC': 'North', 'AP': 'North', 'AM': 'North', 'PA': 'North', 'RO': 'North', 'RR': 'North', 'TO': 'North',
    'AL': 'Northeast', 'BA': 'Northeast', 'CE': 'Northeast', 'MA': 'Northeast', 'PB': 'Northeast', 'PE': 'Northeast', 'PI': 'Northeast', 'RN': 'Northeast', 'SE': 'Northeast',
    'GO': 'Central-West', 'MT': 'Central-West', 'MS': 'Central-West', 'DF': 'Central-West',
    'ES': 'Southeast', 'MG': 'Southeast', 'RJ': 'Southeast', 'SP': 'Southeast',
    'PR': 'South', 'RS': 'South', 'SC': 'South'}

In [4]:
# Loading Data
data_dict = load_data()
olist_customers_df = data_dict['olist_customers_df']
olist_geolocation_df = data_dict['olist_geolocation_df']
olist_order_items_df = data_dict['olist_order_items_df']
olist_order_payments_df = data_dict['olist_order_payments_df']
olist_order_reviews_df = data_dict['olist_order_reviews_df']
olist_orders_df = data_dict['olist_orders_df']
olist_products_df = data_dict['olist_products_df']
olist_sellers_df = data_dict['olist_sellers_df']
product_category_name_translation_df = data_dict['product_category_name_translation_df']
olist_closed_deals_df = data_dict['olist_closed_deals_df']
olist_marketing_qualified_leads_df = data_dict['olist_marketing_qualified_leads_df']

Rating                           860
product_category_name_encoded      0
freight_value                      0
distance_km                        0
month                              0
is_delivery_late                   0
dtype: int64

In [ ]:
df = merge_all_datasets(olist_customers_df, 
                       olist_geolocation_df,
                       olist_order_items_df,
                       olist_order_payments_df,
                       olist_order_reviews_df, 
                       olist_orders_df,
                       olist_products_df, 
                       olist_sellers_df
                       )

In [ ]:
handle_missing = HandleMissingValues()
preprocess_helper = Preprocess()
df = handle_missing.dropping_values(df)
df = handle_missing.geolocation_imputation(df)
df = handle_missing.dropping_columns(df)
df = preprocess_helper.preprocessing(df, state_to_region)
df = preprocess_helper.rolling_mean_process(df)
df_final = preprocess_helper.df_final(df)
df_final.head()

## EDA

In [13]:
#Avantika and Anna's territory

## Imputing is_delivery_late

- The column late_delivery_in_days is calculated as the difference in days between the actual delivery date and the estimated delivery date

- there're 1264 missing values in the late_delivery_in_days column and they are missing because the order was never delivered: order_delivered_customer_date has the missing values

- This brings the question of should these be part of the control, treatent or should simply get dropped?

    1. Definitely not part of the control (on time delivery) as the order was never delivered
    
    2. Not sure about the treatment group, were they late?